<a href="https://colab.research.google.com/github/defeng1/asl-assistant/blob/back-end/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

from google.colab import drive

In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
IMAGE_SIZE = (200, 200)
batch_size = 32

In [6]:
#Load the training, validation and test data sets
train_dir = "/content/gdrive/My Drive/Capstone/Dataset/demo_training_set"
test_dir = "/content/gdrive/My Drive/Capstone/Dataset/demo_test"

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=IMAGE_SIZE,
  batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=IMAGE_SIZE,
  batch_size=batch_size
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_dir,
  image_size=IMAGE_SIZE,
  batch_size=batch_size
)

class_names = train_ds.class_names

print("Successfully loaded train and test set")
print('Number of training batches: %d' % tf.data.experimental.cardinality(train_ds))
print('Number of validation batches: %d' % tf.data.experimental.cardinality(val_ds))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_ds))

Found 26100 files belonging to 29 classes.
Using 20880 files for training.
Found 26100 files belonging to 29 classes.
Using 5220 files for validation.
Found 2900 files belonging to 29 classes.
Successfully loaded train and test set
Number of training batches: 653
Number of validation batches: 164
Number of test batches: 91


In [10]:
#Run this block to load images from disk without having I/O become blocking
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_dir = test_ds.prefetch(buffer_size=AUTOTUNE)

In [11]:
#Run this block for adding artificial training data variations to reduce
#overfitting for smaller datasets
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [12]:
#Preprocessing input for compatibility with MobileNetV2 Model
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
#or
# rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [14]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMAGE_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


9412608/9406464 [==============================] - 0s 0us/step


In [16]:
# image_batch, label_batch = next(iter(train_ds))
# feature_batch = base_model(image_batch)
# print(feature_batch.shape)

(32, 7, 7, 1280)


In [ ]:
# base_model.trainable = False
# base_model.summary()

In [22]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

predition_layer = tf.keras.Sequential([
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(len(class_names))
])

prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

inputs = tf.keras.Input(shape=(200, 200, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

(32, 1280)
(32, 29)


In [24]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 200, 200, 3)       0         
_________________________________________________________________
tf_op_layer_RealDiv (TensorF [(None, 200, 200, 3)]     0         
_________________________________________________________________
tf_op_layer_Sub (TensorFlowO [(None, 200, 200, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_4 ( (None, 1280)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)             

In [ ]:
# Training the new model
initial_epochs = 2

loss0, accuracy0 = model.evaluate(val_ds)
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

history = model.fit(train_ds,
                    epochs=initial_epochs,
                    validation_data=val_ds)

# model.save("directory to save")

In [ ]:
#Run to evaluate model on the test set
test_loss, test_acc = model.evaluate(test_ds, verbose=1)
print("Trained model accuracy {}".format(test_acc))

# loaded_model = tf.keras.models.load_model("saved directory")
# test_loss, test_acc = loaded_model.evaluate(test_ds, verbose=1)
# print("Loaded trained model accuracy {}".format(test_acc))

In [ ]:
#Run to show learning curve of the model
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()